In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from time import sleep
import pandas as pd
from re import sub
from decimal import Decimal
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.webdriver.chrome.options import Options

In [2]:
# specifies the path to the chromedriver.exe
otherdriver = webdriver.Chrome()
# driver.get method() will navigate to a page given by the URL address
otherdriver.get('https://fbatoolkit.com/')

In [3]:
# specifies the path to the chromedriver.exe
driver = webdriver.Chrome()
# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.amazon.com/Best-Sellers/zgbs')

In [4]:
#Read all the departments
#for number in range(1,42):
    #departments = driver.find_element_by_xpath('/html/body/div[3]/div[2]/div/div[1]/div/div[2]/div/div[2]/ul/ul/li['+str(number)+']/a')
    #print(departments.text)

In [5]:
#for Loop 1-50> 
#click posting> apply critiria> Pass?read posting> 
#read related> apply critiria> Pass?click related posting> apply critiria> Pass?read/store posting> 
#click back
#read Frequently together> apply critiria> Pass?click Frequently together posting> apply critiria> Pass?read/store posting> 
#click back
#read customer viewed> apply critiria> Pass?click customer viewed posting> apply critiria> Pass?read/store posting> 
#click back
#click back> increment loop 
#loop finish click next page> repeat loop 

#critiria :frigile[glasses,Plates,Mirrors], price[20-50], weight[<2lb], size[], electrical[watt,voltage,power], 
#on skin[skin], in body[], review[<75],

#critiria test sequence: price[20-50]> title[glasses,Plates,Mirrors,watt,voltage,power,skin]> 
                        #description[glasses,Plates,Mirrors,watt,voltage,power,skin]]> 
                        #product information[glasses,Plates,Mirrors,watt,voltage,power,skin]]
                        #review[<75]> weight[<2lb]> size> 

#read/store :url, title, price, weight, size, number of other sellers, BSR, number of reviews, stars

#size:Maximum weights and dimensions for packaged items
#Product size tier	Weight	Longest side	Median side	Shortest side	Length + girth

#Small standard-size	12 oz	15 inches	12 inches	0.75 inch	n/a

In [4]:
def getjudgeprice():
    goodprice=0
    buyingoptions=0
    try:
        try:
            try:
                pricexpath="//td[text()[contains(.,'Price')]]//following-sibling::*"
                price= driver.find_element_by_xpath(pricexpath)
                price=price.text
                price=price.split()
                price = float(price[0].replace('$', ''))
            except:            
                pricexpath="//li[contains(@class,'swatchAvailable') and contains(@id,'size_name_1')]"
                options= driver.find_element_by_xpath(pricexpath).is_enabled()
                price= driver.find_element_by_xpath(pricexpath)
                price=price.text
                price=price.split('$')
                price=float(price[1])
        except:
            try:
                buyingoptionsxpath="//*[text()[contains(.,'See All Buying Options')]]"
                buyingoptions=driver.find_element_by_xpath(buyingoptionsxpath).is_enabled()
            except:            
                pricexpath="//td[text()[contains(.,'Deal')]]//following-sibling::*"
                price= driver.find_element_by_xpath(pricexpath)
                price=price.text
                price=price.split()
                price = float(price[0].replace('$', ''))
    except:
        try:
            pricexpath="//td[text()[contains(.,'Price')]]//following-sibling::*//child::span[contains(text(),'$')]"
            price= driver.find_element_by_xpath(pricexpath)
            price=price.text
            price=price.split()
            price = float(price[0].replace('$', ''))
        except:
            price =59.59
    if buyingoptions ==1:
        price= 59.59
    if  price >= 60:
        print(price,"is too high")
    if  20 >= price:
        print(price,"is too low")
    if  20 <= price <= 60:
        goodprice=1
    return goodprice,price;

In [5]:
def getjudgetext():
    goodtext=0
    try:
        try:
            titlexpath='/html/body/div[2]/div[1]/div[7]/div[5]/div[3]/div[1]/div/h1/span[1]'
            title=driver.find_element
            title=driver.find_element_by_xpath(titlexpath)
        except:
            titlexpath="//span[contains(@class,'a-size-large') and contains(@id,'productTitle')]"
            title=driver.find_element_by_xpath(titlexpath)            
        title=title.text
        try:
            descriptionxpath='/html/body/div[2]/div[1]/div[7]/div[5]/div[3]/div[33]'
            description=driver.find_element_by_xpath(descriptionxpath)
        except:
            descriptionxpath="//*[contains(concat(' ', normalize-space(@class), ' '), ' feature') and contains(concat(' ', normalize-space(@id), ' '), 'bullet')]"
            description=driver.find_element_by_xpath(descriptionxpath)
        description=description.text
        try:
            try:
                path="//*[contains(text(),'Package') and contains(text(),'Dimensions')]//following-sibling::span"
                productinformation=driver.find_element_by_xpath(path)
                productinformation=productinformation.text
            except:
                productinformationxpath="//div[contains(@id,'detail-bullets_feature_div') and contains(., 'Product details')]"
                productinformation=driver.find_element_by_xpath(productinformationxpath)
                productinformation=productinformation.text
        except:
            try:
                productinformationxpath="//div[contains(@class,'a-section') and contains(@id,'prodDetails')]"
                productinformation=driver.find_element_by_xpath(productinformationxpath)
                productinformation=productinformation.text
            except:
                productinformation='NA'
    except:
        productinformation='NA'
        description='NA'
        title='NA'
    badwords=['Glasses','Glass','Mirror','Watt','Voltage','Power','Skin','glass','plates','mirror','watt','voltage','power','skin','GLASSES',
'GLASS','MIRROR','WATT','VOLTAGE','POWER','SKIN''shoe','shirt','short','shorts','pant','pullover','sweatshirt','hoodie','leggings',
'pants','filter','charger','battery','batteries'
'SHOE','SHIRT','SHORT','SHORTS','PANT','PULLOVER','SWEATSHIRT','HOODIE','LEGGINGS','PANTS','FILTER','CHARGER','BATTERY','BATTERIES'
'Shoe','Shirt','Short','Shorts','Pant','Pullover','Sweatshirt','Hoodie','Leggings','Pants','Filter','Charger','Battery','Batteries']
    if any(word in title for word in badwords) | any(word in description for word in badwords) | any(word in productinformation for word in badwords):
        print(title+'contains bad words')
        splittitle=title.split()
        splitdescription=description.split()
        splitproductinformation=productinformation.split()
        appendedsplit=splittitle+splitdescription+splitproductinformation
        for word in appendedsplit:
            #print(word)
            if word in badwords:
                print(word)
            #else:
                #print('error')
    if not any(word in title for word in badwords) | any(word in description for word in badwords) | any(word in productinformation for word in badwords):
        goodtext=1
    return title,description,productinformation,goodtext;


In [6]:
def getjudgereviews():
    goodreviewcount=0
    try:
        countreviewxpath="//span[contains(@class,'a-size-base') and contains(@id,'acrCustomerReviewText') and contains(., 'ratings')]"
        countreview=driver.find_element_by_xpath(countreviewxpath)
        countreview=countreview.text
        countreview=countreview.replace(' ratings', '')
        countreview=float(countreview.replace(",",''))
        if  countreview <= 75000:
            goodreviewcount=1
    except:
        counterview =30
        goodreviewcount=1
    return countreview,goodreviewcount;

In [7]:
def getjudgeweight():
    shippingweightstring='Shipping Weight'
    try:
        try:
            path="//th[text()[contains(.,'"+str(shippingweightstring)+"')]]//following-sibling::*"
            weight=driver.find_element_by_xpath(path)
            weight=weight.text
            weight=weight.split()
            weightvalue=float(weight[0])
            weightmeasure=weight[1]
            goodweight=0
        except:
            path="//th[text()[contains(.,'"+str(shippingweightstring)+"')]]"
            weight=driver.find_element_by_xpath(path)
            weight=weight.text
            weight=weight.split()
            weightvalue=float(weight[0])
            weightmeasure=weight[1]
            goodweight=0
    except:
        try:
            try:
                path="//*[contains(text(),'Shipping') and contains(text(),'Weight')]//following-sibling::span"
                weight=driver.find_element_by_xpath(path)
                weight=weight.text
                weight=weight.split()
                weightvalue=float(weight[0])
                weightmeasure=weight[1]
                goodweight=0
            except:
                try:
                    path="//*[contains(text(),'Product details')]//parent::*//*[contains(text(),'pounds')]"
                    weight=driver.find_element_by_xpath(path)
                    weight=weight.text
                    numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
                    rx = re.compile(numeric_const_pattern, re.VERBOSE)
                    weightsplit=rx.findall(weight)
                    weightvalue=float(weightsplit[-1])
                    weightmeasure='pounds'
                    goodweight=0
                except:
                    path="//*[contains(text(),'Product details')]//parent::*//*[contains(text(),'ounce')]"
                    weight=driver.find_element_by_xpath(path)
                    weight=weight.text
                    numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
                    rx = re.compile(numeric_const_pattern, re.VERBOSE)
                    weightsplit=rx.findall(weight)
                    weightvalue=float(weightsplit[-1])
                    weightmeasure='ounce'
                    goodweight=0
        except:
            try:
                path="//*[contains(text(),'Weight') or contains(text(),'weight')]//parent::tr//following-sibling::td"
                weight=driver.find_element_by_xpath(path)
                weight=weight.text
                weight=weight.split()
                weightvalue=float(weight[0])
                weightmeasure=weight[1]
                goodweight=0
            except:    
                weightvalue=1
                goodweight=0
                weightmeasure='ounce'
            


    #weight=weight.replace(' (View shipping rates and policies)', '')

    if weightvalue <= 2:
            goodweight=1
    if 'ounce' in weightmeasure:
        if weightvalue <= 15:
            goodweight=1
    if goodweight==0:
        print(weightvalue,'',weightmeasure,'is too heavy')
    if goodweight==1:
        goodweight==1
    return weightvalue,weightmeasure,goodweight;


In [8]:
def getjudgedimensions():
    goodsize=0
    try:
        ProductDimensions='Dimensions'
        try:
            try:
                path="//*[text()[contains(.,'"+str(ProductDimensions)+"')]]//following-sibling::*"
                size=driver.find_element_by_xpath(path)
                size=size.text
            except:
                path="//*[text()[contains(.,'"+str(ProductDimensions)+"')]]"
                size=driver.find_element_by_xpath(path)
                size=size.text
            measure1=float(sizesplit[0])
            measure2=float(sizesplit[2])
            measure3=float(sizesplit[4])
            sizemeasureunit=sizesplit[5]  
        except:
            try:
                try:
                    path="//*[contains(text(),'Package') and contains(text(),'Dimensions')]//following-sibling::span"
                    productinformation=driver.find_element_by_xpath(path)
                    productinformation=productinformation.text
                except:
                    productinformationxpath="//div[contains(@id,'detail-bullets_feature_div') and contains(., 'Product details')]"
                    productinformation=driver.find_element_by_xpath(productinformationxpath)
                    productinformation=productinformation.text
                    sizewords=['inche','Inche','inches','Inches','cm','centimeter','Centimeter']
                    if any(word in productinformation for sizewords in inche):
                        inche=['inche','Inche','inches','Inches']
                        if any(word in productinformation for word in inche):
                            sizemeasureunit='inches'
                        else:
                            sizemeasureunit='cm'
                    else:
                        float(1)
            except:
                try:
                    productinformationxpath="//div[contains(@class,'a-section') and contains(@id,'prodDetails')]"
                    productinformation=driver.find_element_by_xpath(productinformationxpath)
                    productinformation=productinformation.text
                except:
                    try:
                        path="//*[contains(text(),'Dimensions')]//following-sibling::td"
                        productinformation=driver.find_element_by_xpath(path)
                        productinformation=productinformation.text
                    except:
                        path="//*[contains(text(),'inches')]"
                        productinformation=driver.find_element_by_xpath(path)
                        productinformation=productinformation.text
            goodsize=0
            numeric_const_pattern = '[-+]? (?: (?: \d* \. \d+ ) | (?: \d+ \.? ) )(?: [Ee] [+-]? \d+ ) ?'
            rx = re.compile(numeric_const_pattern, re.VERBOSE)
            sizesplit=rx.findall(productinformation)
            sizesplit
            try:
                measure1=float(sizesplit[0])
                measure2=float(sizesplit[1])
                measure3=float(sizesplit[2])
                inche=['inche','Inche','inches','Inches']
                if any(word in productinformation for word in inche):
                    sizemeasureunit='inches'
                else:
                    sizemeasureunit='cm'
            except:
                measure1=float(sizesplit[1])
                measure2=float(sizesplit[3])
                measure3=float(sizesplit[5])
                sizemeasureunit=sizesplit[6]            
        longest = max(measure1, measure2, measure3)
        shortest = min(measure1, measure2, measure3)
        median=measure1+measure2+measure3
        median=median-shortest-longest
        if longest<=18 and shortest<=8  and  median<=14 :
            goodsize=1
        else:
            print(size,"is too big")
    except:
        longest=2
        shortest=2
        median=2
        goodsize=1
    return longest,shortest,median,goodsize;

In [9]:
def frequentbuywith():
    freqavailable=0
    frequrl0='NA'
    frequrl1='NA'
    frequrl2='NA'
    try:
        path = "//div[contains(@id,'sims-fbt-container')]//a[@class='a-size-base a-link-normal']"
        urlfreq = driver.find_elements_by_xpath(path)
        frequrl0=urlfreq[0].get_attribute("href")
        freqavailable=1
        try:
            path = "//div[contains(@id,'sims-fbt-container')]//a[@class='a-size-base a-link-normal']"
            urlfreq = driver.find_elements_by_xpath(path)
            frequrl1=urlfreq[1].get_attribute("href")
            freqavailable=1 
        except:
            print('only one frequent')
           
    except:
        print('no frequent buying')  
            

    return frequrl0,frequrl1,freqavailable;

In [10]:
def relateditems():
    urlemptylist = []
    titlelist = []
    element = driver.find_element_by_xpath("//*[contains(text(),'Sponsored') and contains(text(),'product') and contains(text(),'related')]")
    driver.execute_script("arguments[0].scrollIntoView(true);",element)
    try:

            #get links
            #path = "//li[contains(@class,'a-carousel-card')]//following-sibling::a[contains(@class,'a-link-normal') and contains(@target,'_top')]"
            #elementsrelated = driver.find_elements_by_xpath(path)
            #for related in elementsrelated:
            #    relatedurl=related.get_attribute("href")
        for subpage in range(0,3):
            #get titles and judge
            path="//*[contains(text(),'Sponsored') and contains(text(),'product') and contains(text(),'related')]//parent::div/parent::div/parent::div//following-sibling::div[@class='sponsored-products-truncator-truncated']"
            titleselementsrelated=driver.find_elements_by_xpath(path)
            for title in range(0,6):
                relatedtitle=titleselementsrelated[title].text
                print(relatedtitle)
                badwords=['Glasses','Mirror','Watt','Voltage','Power','Skin','glasses','mirror','watt','voltage','power','skin','GLASSES','MIRROR',
        'WATT','VOLTAGE','POWER','SKIN''shoe','shirt','short','shorts','pant','pullover','sweatshirt','hoodie','leggings','pants','filter','charger','battery','batteries'
        'SHOE','SHIRT','SHORT','SHORTS','PANT','PULLOVER','SWEATSHIRT','HOODIE','LEGGINGS','PANTS','FILTER','CHARGER','BATTERY','BATTERIES'
        'Shoe','Shirt','Short','Shorts','Pant','Pullover','Sweatshirt','Hoodie','Leggings','Pants','Filter','Charger','Battery','Batteries']
                goodtext=0
                #get and judge title,description and product information
                if any(word in relatedtitle for word in badwords) :   
                    splittitle=relatedtitle.split()
                    for word in splittitle:
                        #print(word)
                        if word in badwords:
                            print(word)
                    #print('related '+relatedtitle+' contains bad words')
                if not any(word in relatedtitle for word in badwords) :
                    goodtext=1
                if goodtext==1:
                    #get price and judge price 
                    path="//*[contains(text(),'Sponsored') and contains(text(),'product') and contains(text(),'related')]//parent::div/parent::div//following-sibling::div//span[@class='a-color-price']"
                    pricerelated=driver.find_elements_by_xpath(path)
                    price=pricerelated[title].text
                    price=price.split('$')
                    price=float(price[1])
                    goodprice=0
                    #if  price >= 60:
                        #print(price,"is too high")
                    #if  20 >= price:
                        #print(price,"is too low")
                    if  20 <= price <= 60:
                        goodprice=1
                        #print("price is good")
                    if goodprice==1 :
                        #get reviews and judge reviews
                        path="//*[contains(text(),'Sponsored') and contains(text(),'product') and contains(text(),'related')]//parent::div/parent::div/parent::div//following-sibling::span[@class='a-color-link']"
                        reviewselements=driver.find_elements_by_xpath(path)
                        countreview=reviewselements[title].text
                        countreview=float(countreview.replace(",",''))
                        goodreviewcount=0
                        if  countreview <= 750000:
                            goodreviewcount=1
                        if goodreviewcount==1:
                            #print('reviews is good')
                            #add item url to a list
                            currenturl= driver.current_url
                            path="//*[contains(text(),'Sponsored') and contains(text(),'product') and contains(text(),'related')]//parent::div/parent::div/parent::div//following-sibling::a[@class='a-link-normal']"
                            urlelementsrelated=driver.find_elements_by_xpath(path)
                            relatedurl=urlelementsrelated[title].get_attribute("href")
                            urlemptylist.append(relatedurl)
                            print('added to list')

            #press pagify button for related items 
            path="//*[contains(text(),'Sponsored') and contains(text(),'product') and contains(text(),'related')]//parent::div/parent::div/parent::div//span[@class='a-button-inner']"
            relatednextpage=driver.find_elements_by_xpath(path)
            relatednextpage[1].click()
            sleep(1)

        myList=[]        
        currenturl= driver.current_url
        i=0
        print('reading from potential url list')
        for url in range(len(urlemptylist)):
            driver.get(urlemptylist[url])
            sleep(2)
            current = driver.current_url
            #get and judge price
            goodprice, price = getjudgeprice()
            print('judged price')
            if goodprice==1:
                print('price is good')
                #get and judge title,description and product information
                title,description,productinformation, goodtext = getjudgetext()
                print('judged text')
                if goodtext==1:
                    print('text is good')
                    #judge and get weight
                    weightvalue,weightmeasure,goodweight=getjudgeweight()
                    print('judged weight')
                    if goodweight==1:
                        print('weight is good')
                        #judge and get dimensions
                        longest,shortest,median,goodsize=getjudgedimensions()
                        print('dimensions are good')
                        print('judged dimensions')
                        if goodsize==1:
                            print('****seems good')
                            i=i+1
                            rank=getrank()
                            sales=getmonthlysale(rank)
                            myList.append([current,price,title,description,productinformation,countreview,weightvalue,weightmeasure,longest,shortest,median,rank,sales])

    except:
        print('could not get any related items')
        myList=[]
    
    return myList,urlemptylist;

In [11]:
def find_str(s, char):
    index = 0
    if char in s:
        c = char[0]
        for ch in s:
            if ch == c:
                if s[index:index+len(char)] == char:
                    return index
            index += 1
    return -1
def getrank():
    try:
        path="//*[contains(text(),'Rank')]//parent::*"
        rank=driver.find_element_by_xpath(path)
        rank=rank.text
        index=find_str(rank, "#")
        rank=rank[index:index+10]
        rank=rank.split()
        ranknumber=rank[0].split('#')
        return int(ranknumber[1].replace(',', ''));
    except:
        print('giving dummy rank')
        return 9999;

In [12]:
def getmonthlysale(rank):
    sleep(1)
    inputpath="/html/body/div[2]/div[6]/div[4]/div[17]/div[3]/div/input[1]"
    rankinput=otherdriver.find_element_by_xpath(inputpath)    
    rankinput.click()
    rankinput.send_keys(rank)
    gopath="/html/body/div[2]/div[6]/div[4]/div[17]/div[3]/div/input[2]"
    go=otherdriver.find_element_by_xpath(gopath)
    go.click()
    sleep(2)
    valuepath="/html/body/div[2]/div[6]/div[4]/div[17]/div[3]/div/span[2]/div[1]"
    value=otherdriver.find_element_by_xpath(valuepath)
    value=value.text
    otherdriver.find_element_by_xpath(inputpath).clear()
    return value;

In [19]:
myList = []
for deppage in range(1,3):
    sleep(2)
    dep_page_url = driver.current_url
    #click 1-50 1by1 the departments
    for posting in range(1,51):
        try:
            print('reading post number',posting)
            postingtitle = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div/div[1]/div/ol/li['+str(posting)+']/span/div/span/a/div')
            postingtitle.click()
            sleep(2)
            top100producturl = driver.current_url
            #get and judge price
            goodprice, price = getjudgeprice()
            if goodprice==1:
                #get and judge title,description and product information
                title,description,productinformation, goodtext = getjudgetext()
                if goodtext==1:    
                    #judge and get reviews
                    countreview,goodreviewcount=getjudgereviews()
                    if goodreviewcount==1:
                        #judge and get weight
                        weightvalue,weightmeasure,goodweight=getjudgeweight()
                        if goodweight==1:
                            #judge and get dimensions
                            longest,shortest,median,goodsize=getjudgedimensions()
                            if goodsize==1:
                                print('----seems good')
                                rank=getrank()
                                sales=getmonthlysale(rank)
                                myList.append([top100producturl,price,title,description,productinformation,countreview,weightvalue,weightmeasure,longest,shortest,median,rank,sales])
            #check frequent
            mainbestsellerurl=driver.current_url
            frequrl0,frequrl1,freqavailable = frequentbuywith()
            if freqavailable==1:
                freqitem=driver.get(frequrl0)
                sleep(2)
                #get and judge price
                goodprice, price = getjudgeprice()
                if goodprice==1:
                    #get and judge title,description and production information
                    title,description,productioninformation, goodtext = getjudgetext()
                    if goodtext==1:    
                        #judge and get reviews
                        countreview,goodreviewcount=getjudgereviews()
                        if goodreviewcount==1:
                            #judge and get weight
                            weightvalue,weightmeasure,goodweight=getjudgeweight()
                            if goodweight==1:
                                #judge and get dimensions
                                longest,shortest,median,goodsize=getjudgedimensions()
                                if goodsize==1:
                                    print('freq1 seems good')
                                    rank=getrank()
                                    sales=getmonthlysale(rank)
                                    myList.append([frequrl0,price,title,description,productioninformation,countreview,weightvalue,weightmeasure,longest,shortest,median,rank,sales])                                              
                if 'NA' not in frequrl1:
                    freqitem=driver.get(frequrl1)
                    sleep(2)
                    #get and judge price
                    goodprice, price = getjudgeprice()
                    if goodprice==1:
                        #get and judge title,description and production information
                        title,description,productinformation, goodtext = getjudgetext()
                        if goodtext==1:    
                            #judge and get reviews
                            countreview,goodreviewcount=getjudgereviews()
                            if goodreviewcount==1:
                                #judge and get weight
                                weightvalue,weightmeasure,goodweight=getjudgeweight()
                                if goodweight==1:
                                    #judge and get dimensions
                                    longest,shortest,median,goodsize=getjudgedimensions()
                                    if goodsize==1:
                                        print('freq2 seems good')
                                        rank=getrank()
                                        sales=getmonthlysale(rank)
                                        myList.append([frequrl1,price,title,description,productinformation,countreview,weightvalue,weightmeasure,longest,shortest,median,rank,sales])
            #check related
            try:
                driver.get(mainbestsellerurl)
                actions = ActionChains(driver)
                for _ in range(8):
                    actions.send_keys(Keys.PAGE_DOWN).perform()
                    time.sleep(1)
                sleep(3)
                listofgood,i=relateditems()
                myList=myList+listofgood
            except:
                print('can not get related')
            #cheack also viewed

            #back to department page
            driver.get(dep_page_url)
            sleep(2)
            print(str(posting)+' done')
        except:
            print('########product number '+str(posting)+' not found')
            #back to department page
            driver.get(dep_page_url)
            sleep(2)
    nextpage=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div/div/div[1]/div/div[2]/div/ul/li[4]")
    nextpage.click()

reading post number 1
14.59 is too low
Diapers Size 3, 168 Count - Pampers Swaddlers Disposable Baby Diapers, ONE MONTH SUPPLYcontains bad words
skin
Diapers Size 2, 186 Count - Pampers Swaddlers Disposable Baby Diapers, ONE MONTH SUPPLYcontains bad words
skin
WaterWipes Sensitive Baby Wipes, 720 Count (12 Packs of 60 Count)
added to list
Huggies Little Snugglers Baby Diapers, Size 5, 120 Ct, One Month Supply
added to list
Huggies Little Movers Baby Diapers, Size 4, 144 Ct, One Month Supply
added to list
Baby Wipes, Pampers Aqua Pure Sensitive Water Baby Diaper Wipes, Hypoallergenic and...
added to list
Mark total wipes Baby Natural Sensitive Baby Wipes 800 Count (10 Pack of 80 Count) ...
added to list
Aveeno Baby Sensitive All Over Wipes with Aloe & Natural Oat Extract for Face, Bott...
100% Pure Cotton Dry Wipes | 600 Count | Use Wet or Dry | Soft & Sensitive | Hypoal...
added to list
Diapers Size 4, 150 Count - Pampers Pure Protection Disposable Baby Diapers, Hypoal...
added to list

Mr. Clean Magic Eraser Kitchen, Cleaning Pads with Durafoam, 4 count (Packaging May...
Magic Eraser by Mr. Clean Variety Pack, Cleaning Pads with Durafoam, 6 Count, Pack ...
Bottle Brush Cleaner 5 Pack - Long Water Bottle and Straw Cleaning Brush - Kitchen ...
Comotomo Natural Feel Baby Bottle, Green, 5 Ounce
Bathroom Surfaces Tub, Shower, Tile and Grout All Purpose Power Scrubber Cleaning Kit
Power
Dr. Brown's Baby Bottle, Options+ Anti-Colic Narrow Bottle, 8 Ounce (Pack of 4)
added to list
Bottle Brush Cleaner 5 Pack - Long Water Bottle and Straw Cleaning Brush - Kitchen ...
Dr. Brown's Deluxe Bottle Sterilizer
added to list
Dr. Brown's Microwave Steam Sterilizer
added to list
Tommee Tippee Closer to Nature Portable Travel Baby Bottle Warmer - Multi Function -  BPA Free
Set of Bottle Cleaning Brushes 3Pcs Kitchen Nylon Multifunction Clean Brush Set (L.&G.)
MoomooBaby Triple Brush Set - Baby Bottle Cleaner Brush, Nipple Brush & Straw Cleaner
Long Bottle Brush Cleaner Set (3-in-1) and 

Wall Nanny Mini - Smallest Baby Gate Wall Protector - Low-Profile (Made in USA) for Dog & Pet Gates…
Sungrace Y Spindle Banister Gate Adaptors for Pressure Mounted Baby Gates (2 Pack, 10mm)
reading from potential url list
judged price
price is good
judged text
text is good
9.75  pounds is too heavy
judged weight
7 done
reading post number 8
4.99 is too low
3.64 is too low
5.49 is too low
Clear Outlet Covers Baby Proofing - Vmaisi 38 Pack Electrical Safety ChildProof Plu...
Outlet Plug Covers (32 Pack) Clear Child Proof Electrical Protector Safety Caps - J...
Clear Outlet Covers – Value Pack 50 Count Premium Quality – New & Improved Baby Saf...
36 Pack, Outlet Covers ChildProof Plug Protector, Safe & Secure Baby Proofing Elect...
Outlet Covers Babyproofing 50-Pack by Wappa Baby | Safe & Secure Electric Plug Prot...
Outlet Covers ChildProof Plug Protector - Vmaisi Baby Proofing Electrical Safety Ou...
Baby Proofing Outlet Covers with Hidden Pull Handle (40 Pack) Keep Your Kids and Pe...


reading from potential url list
judged price
price is good
judged text
text is good
judged weight
weight is good
dimensions are good
judged dimensions
****seems good
judged price
price is good
judged text
text is good
judged weight
weight is good
dimensions are good
judged dimensions
****seems good
11 done
reading post number 12
6.99 is too low
12.99 is too low
9.99 is too low
Brother Max Baby Snack Bowl with Lid, Snack Catcher for Toddler, Ideal for Travel, ...
OXO Tot Water Bottle & Straw Cup Cleaning Set
OXO Tot Baby Blocks Freezer Storage Containers (4 Oz)
OXO Tot 12-Piece Baby Blocks Set, Teal
Brother Max Baby Feeding Bowl with Lid Set for Toddler, Baby Divided Bowl with Spoo...
Silikong Suction Plate for Toddlers | BPA Free | Microwave, Dishwasher and Oven Saf...
Suction Baby Bowls for Kids Toddlers Solid Feeding, Cute Stone 3 Size Stay Put Spil...
Brother Max Baby Milk Powder Formula Dispenser Snack Cup, Blue/Green.
LittleHippo Mella Ready to Rise Children's Trainer, Alarm Clock

judged price
price is good
Flushable Baby Wipes for Kids, Sensitive by Kandoo, Hypoallergenic Potty Training Wet Cleansing Cloths Refills, Unscented, 250 Count per Pack, Pack of 4contains bad words
judged text
judged price
price is good
judged text
text is good
12.4  pounds is too heavy
judged weight
judged price
price is good
judged text
text is good
3.7  pounds is too heavy
judged weight
judged price
price is good
Diapers Newborn / Size 0 (< 10 lb), 92 Count - Pampers Pure Disposable Baby Diapers, Hypoallergenic and Unscented Protection, Giant Packcontains bad words
skin
judged text
judged price
price is good
judged text
text is good
14.05  pounds is too heavy
judged weight
14 done
reading post number 15
17.67 is too low
no frequent buying
Baby Nasal Aspirator with 24 Filters, Infant Premium Mucus Aspirator Snot Sucker, B...
BabyComfy Nasal Aspirator -- The Snotsucker -- Hygienically & Safely Removes Baby’s...
Baby Nasal Aspirator Nose Sucker - BORIOS USB Rechargeable Electric Nose S

...

Organic Nipple Butter Breastfeeding Cream by Earth Mama | Lanolin-free, Safe for Nursing & Dry…
Hands-Free Pumping Bra, Nursing Bra, Wireless, Adjustable Zipper Breastfeeding Bra for…
reading from potential url list
judged price
price is good
Lausana Nursing Cover for Breastfeeding Mulit-Use Shawl Wrap Stole for Women (Pink)contains bad words
skin
judged text
judged price
price is good
NOONI'S Revolutionary Nursing Bra That No Longer Requires Breast Padscontains bad words
shirt
shirt
judged text
judged price
price is good
tntnmom's - Cabbage Breast Cream for Nursing - Breastfeeding Trouble Relief & Good for Breast Pain and Mastitis | Breast Feeding Essentials | Lanolin-Free | Korean Skin Care | 3.38 Fl ozcontains bad words
Skin
skin
skin
judged text
judged price
price is good
Haakaa Breast Shells Milk Savers for Breastfeeding Breast Milk Catcher Soft Flexible Silicone Nipple Shells Reusable and Easy to Wear, 2 Packcontains bad words
judged text
judged price
price is good
Lausana Nursin

Toddler Bed Rail Guard for Kids Twin, Double, Full Size Queen & King Mattress - Bed...
added to list
Baby BBZ 59inch Bed Rail - Single Foldable Safety Bedrail with Ventilated Mesh for ...
added to list
[2-Pack] hiccapop Inflatable Bed Rail for Toddlers | Travel Bed Rail, Blow-up Bed B...
added to list
Modvel Toddler Bed Bumper Rail Guard [1-Pack] | Comfortable Hypoallergenic Foam for...
added to list
Toddler Bed Rail Guard for Kids Twin, Double, Full Size Queen & King Mattress - Bed Rails for Toddlers…
added to list
(2-Pack) Extra-Tall Foam Bed Rails for Toddler | Soft Bed Bumpers for Kids | Baby Bed Guard | Child Bed…
added to list
Milliard Bed Bumper (with Attached Sheet) for Toddler Bed, Foam Safety Rail Guard with Washable…
added to list
Modvel Toddler Bed Bumper Rail Guard [2-Pack] | Comfortable Hypoallergenic Foam for…
added to list
(2-Pack) DreamyPanda Toddler Bed Rail Bumper/Foam Guard for Bed - Large - Side Rail…
added to list
SURPCOS 70" Bed Rails for Toddlers - Extra Long Ba

added to list
35 Packs One Box Baby Bathing Diaper Refill Bags with Toss and Hassle Free Blue Bag...
added to list
Upgraded Thicken 50%, Diaper Genie Refill Bags, Compatible with Diaper Genie Pails,...
Momcozy Diaper Disposable Bags, Poop Bags with Tie Handle, Durable and Unscented Portable…
Nuby Diaper Bags, 100 Count (2 Packages)
Baby Disposable Diaper Bags, 100% Biodegradable Diaper Sacks with Baby Powder Scent and Added…
300 Count Baby Disposable Diaper Sacks Bags, Diaper Bags Easy-Tie with Baby Powder Scent
S.A.C. Adult Diaper Disposable Bags 50-Count, Set of 3
added to list
3 Count Dekor Classic Refills by Momcozy | Vanilla Scent | 22 Layers Extra Thickness | Fits Dekor…
BOS Amazing Odor Sealing Baby Disposable Diaper Bags, Also for Pet Waste or Any Sanitary Product…
Baby Disposable Diaper Sacks with Powder Scent, 90 Counts, Disposal Bags Dispenser Easy-Tie
Party Bargains Disposable Diaper Bags with Baby Powder Scent | 100% Biodegradable Easy-Tie…
BodyGuard Baby Diapers Disposal 

Premium Quality Nursing Pillow Cover by Mila Millie - Nordic Swiss Black Cross Unisex Design Slipcover…
Boppy Pregnancy Support Pillow, Petite Trellis Gray and White, Body Pillow with removable jersey cover
added to list
DSTANA Kids Constellation Night Light Star Lamp with Timer Auto-Shut of 360 Degree LED Rotation of…
Premium Quality Nursing Pillow Cover by Mila Millie - Gray Chevron Unisex Design Slipcover - 100% Cotton…
Water Resistant Removable Cover for Newborn Lounger | Unisex Gray Arrow Design | Premium Quality Soft…
added to list
Premium Quality 2 Pack Nursing Pillow Covers by Mila Millie - Nordic Swiss Black & White Unisex…
reading from potential url list
judged price
price is good
NOONI'S Revolutionary Nursing Bra That No Longer Requires Breast Padscontains bad words
shirt
shirt
judged text
judged price
price is good
judged text
text is good
3.8  pounds is too heavy
judged weight
judged price
price is good
judged text
text is good
2.6  pounds is too heavy
judged weight
judged

50 done


In [20]:
applaiances2 = pd.DataFrame(myList, columns=('top100producturl', 'price','title', 'description', 'productinformation', 'countreview', 'weightvalue','weightmeasure','longestest','shortest','median','rank','sales'))
applaiances2

,top100producturl,price,title,description,productinformation,countreview,weightvalue,weightmeasure,longestest,shortest,median,rank,sales
0,https://www.amazon.com/dp/B07DJ1F97S/ref=sspa_...,20.99,Haakaa Manual Breast Pump with Base 5oz/150ml,"Updated suction base and large capacity 150ml,...",NA,53.0,5.60,ounces,7.2,3.3,3.3,10855,2 (30 Days Avg)
1,https://www.amazon.com/dp/B00UV6JDN8/ref=sspa_...,58.99,Freemie Collection Cups The Only Hands Free an...,Compatible pump not included.Compatible with F...,NA,53.0,13.60,ounces,7.8,5.3,5.3,1530,17 (30 Days Avg)
2,https://www.amazon.com/dp/B01845QH7M/ref=sspa_...,22.99,"Dr. Brown's Baby Bottle, Options+ Anti-Colic N...",Anti-colic vent system is clinically proven to...,NA,6068.0,13.60,ounces,1.0,1.0,1.0,183,71 (30 Days Avg)
3,https://www.amazon.com/dp/B009QXDE32/ref=sspa_...,23.97,"Comotomo Baby Bottle, Green, 8 Ounce (2 Count)",Comotomo baby bottles are deisgned to most clo...,NA,6068.0,10.60,ounces,6.0,2.6,5.2,68,71 (30 Days Avg)
4,https://www.amazon.com/dp/B009QXDE32/ref=sspa_...,23.97,"Comotomo Baby Bottle, Green, 8 Ounce (2 Count)",Comotomo baby bottles are deisgned to most clo...,NA,6068.0,10.60,ounces,6.0,2.6,5.2,73,71 (30 Days Avg)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,https://www.amazon.com/dp/B076L4ZWC3/ref=sspa_...,34.99,GentlePack Reusable Perineal Ice & Heat Packs ...,,Product details\nColor: Black\nShipping Weight...,1.0,1.70,pounds,2.0,2.0,2.0,23334,1 (30 Days Avg)
100,https://www.amazon.com/dp/B07GZSWGV8/ref=sspa_...,23.51,Mangrove Tree Naturals Organic Herb Sitz Bath ...,,NA,1.0,5.10,ounce,2.0,2.0,2.0,360718,0 (30 Days Avg)
101,https://www.amazon.com/dp/B01DOE0DO6/ref=sspa_...,24.97,BCOZZY Kids Chin Supporting Travel Pillow- Kee...,PATENTED CHIN SUPPORT- The overlapping arms GI...,Product information\nSize:Child | Color:Navy...,5544.0,2.88,ounces,9.4,0.4,8.3,453,49 (30 Days Avg)
102,https://www.amazon.com/dp/B000I2MRMY/ref=sspa_...,29.99,"LAMAZE, Octotunes, Musical Octopus Stuffed Bab...",MUSICAL OCTOPUS: Make merry music together wit...,NA,995.0,1.15,pounds,13.5,7.8,13.0,13335,2 (30 Days Avg)


In [22]:
applaiances2.to_csv(r'C:\Users\pc\Google Drive\Personal\Work\Online\Jupyter\Git\Amazonwebscraping\Baby.csv')